In [ ]:
import datetime
from dataDownloader  import DataDownloader
import dataDownloader
import yfinance as yf;
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
import pandas as pd

from portfolio import Portfolio

# 나중에 해야할일
# company 데이터 가져올때, get_mark_spec 메서드 활요하면 스펙기업인지 아닌지 확인할 수 있음.
# info는 dict임, dict를 까서 isspec인지 체크하고, 그 딕셔너리에 isSpec인지 체크하고, 여기서 이것도 확인하면 좋나? 재표가 있는지?


# 스코어링 방식
# 기본조건은 다음과 같음
# 1. isSpec 제외
# 2. 상장한지 3년 이상의 데이터 존재. (즉, 현재 기준으로 4년차 데이터가 존재해야함. 2022년 01월부터 데이터 존재)
# 3. 연간 재무재표 3년이상 존재
# 4. 분기 재무재표 4분기이상 존재

# with DB_NYSE() as nyse:
#    symbols = nyse.getSymbolList()
#    display(len(symbols))
      

with DB_FinancialStatement() as fs:
   symbols = fs.getSymbolListByFilter()
   display(len(symbols))


# for symbol in symbols[:5]:
# ticker = yf.Ticker('AACP')
# data = DataDownloader.get_mark_spac_dict(ticker.info)
# display(data)

# DataDownloader.downloadCompanyInfoAndSaveDB(symbols[4270:])
# Portfolio.show_portfolio()


# 상장한 날짜 최초로 알게되는 날짜.
ticker = yf.Ticker('AAPL')
hist = ticker.history(period='max')
date = hist.index[0].date()
print(date)

first_trade_date = ticker.info.get("firstTradeDateMilliseconds")
first_trade_date = datetime.datetime.fromtimestamp(first_trade_date/1000).date()
print(first_trade_date)


4194

1980-12-12
1980-12-12
